In [1]:
import pandas_datareader.data as web
import datetime
import numpy as np
import pandas as pd

In [ ]:
# Define date range
start = datetime.datetime(2002, 5, 28)
end = datetime.datetime(2025, 4, 9)

# Fetch data
vix = web.DataReader('VIXCLS', 'fred', start, end)
two_year_yield = web.DataReader('DGS2', 'fred', start, end)
ten_minus_two = web.DataReader('T10Y2Y', 'fred', start, end)

# Rename columns
vix.rename(columns={"VIXCLS": "VIX"}, inplace=True)
two_year_yield.rename(columns={"DGS2": "2Y_Yield"}, inplace=True)
ten_minus_two.rename(columns={"T10Y2Y": "10Y-2Y_Spread"}, inplace=True)

# Merge data into single DataFrame
data = vix.join([two_year_yield, ten_minus_two])

# Replace zeros with NaN
data.replace(0, np.nan, inplace=True)

# Interpolate missing data linearly
data.interpolate(method='linear', inplace=True)

# Forward/backward fill any remaining NaNs
data.ffill(inplace=True)
data.bfill(inplace=True)

# Convert yields from percentage to decimal
data['2Y_Yield'] = data['2Y_Yield'] / 100
data['10Y-2Y_Spread'] = data['10Y-2Y_Spread'] / 100

# Export to CSV
data.to_csv("new_market_data.csv")

print("CSV file saved as market_data.csv")


CSV file saved as market_data.csv


In [6]:
# Define the start and end dates
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime.today()

# Download the daily U.S. 3-month T-bill rate from FRED (ticker: DTB3)
data = web.DataReader('DTB3', 'fred', start, end)

# The DTB3 data is the annualized yield in percentage.
# Convert to daily return using compounding: (1 + annual_rate/100)^(1/252) - 1
data['daily_return'] = (1 + data['DTB3']/100) ** (1/252) - 1

# Display the last few records
print(data.tail())

            DTB3  daily_return
DATE                          
2025-04-08  4.19      0.000163
2025-04-09  4.24      0.000165
2025-04-10  4.22      0.000164
2025-04-11  4.23      0.000164
2025-04-14  4.22      0.000164


In [10]:
import pandas as pd
import numpy as np
import datetime
from pandas_datareader import data as web

# Define the start and end dates
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime.today()

# Download the daily U.S. 3-month T-bill rate from FRED
data = web.DataReader('DTB3', 'fred', start, end)

# Convert annual rate to daily return
data['daily_return'] = (1 + data['DTB3']/100) ** (1/252) - 1

# Reindex to full daily calendar (including weekends)
full_index = pd.date_range(start=start, end=end, freq='D')
data = data.reindex(full_index)

# Linearly interpolate missing values
data['DTB3'] = data['DTB3'].interpolate(method='linear')
data['daily_return'] = (1 + data['DTB3']/100) ** (1/252) - 1

# Load ETF returns
etf_returns = pd.read_csv('1trading_etf_returns_aligned.csv', parse_dates=['date'])
etf_returns.set_index('date', inplace=True)

# Align and merge 'rf' from interpolated daily_return
etf_returns['rf'] = data['daily_return'].reindex(etf_returns.index)

# Export to CSV
etf_returns.to_csv("2trading_etf_returns_aligned.csv")

print("CSV file saved as 2trading_etf_returns_aligned.csv")




CSV file saved as 2trading_etf_returns_aligned.csv
